In [2]:
import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from preprocess_utility import *
from data_manager import *
from metadata import *

Setting environment for Gordon


No vtk


In [3]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

In [4]:
exclude_nodes = [33, 31]

# Generate score volumes

In [5]:
# for stack in all_stacks:
for stack in ['MD635']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]

#     #################################

#     t = time.time()
#     sys.stderr.write('running svm classifier ...')


#     run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/svm_predict.py',
#                     'stack': stack},
#                     kwargs_list=dict(sections=range(first_sec, last_sec+1)),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='partition')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds

# #     #################################

#     t = time.time()
#     sys.stderr.write('interpolating scoremaps ...')

#     run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/interpolate_scoremaps_v2.py',
#                     'stack': stack},
#                     kwargs_list=dict(sections=range(first_sec, last_sec+1)),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='partition')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # ~240 seconds 

#     #################################

#     t = time.time()
#     sys.stderr.write('visualize scoremaps ...')

#     add_annotation = False
#     # first_sec, last_sec = DataManager.load_cropbox(stack)[4:]

#     run_distributed4(command='%(script_path)s %(stack)s -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/visualize_scoremaps_v2.py',
#                     'stack': stack,
#                     'add_annotation': '-a' if add_annotation else ''},
#                     kwargs_list=dict(sections=range(first_sec, last_sec+1)),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='partition')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack


#     #################################

#     t = time.time()
#     sys.stderr.write('constructing score volumes ...')

#     run_distributed4(command='%(script_path)s %(stack)s %%(label)s' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct') + '/construct_score_volume_v2.py',
#                     'stack': stack},
#                     kwargs_list=dict(label=structures),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='single')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 600 seconds

    #################################

    downscale_factor = 32

    #################################

    print stack

    volume_allLabels = {}

    for name_u in structures:
        volume = DataManager.load_score_volume(stack, label=name_u, downscale=downscale_factor, train_sample_scheme=1)
        volume_allLabels[name_u] = volume
#         del volume

    t1 = time.time()

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))

    for name_u in structures:

        t = time.time()

        gy_gx_gz = np.gradient(volume_allLabels[name_u].astype(np.float32), 3, 3, 3) 
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important; 
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 20s (float32) vs. 2s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (name_u, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[1], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[2], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name_u, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

MD635


Gradient 5N: 2.943852 seconds
save 5N: 3.163096 seconds
Gradient 6N: 2.879673 seconds
save 6N: 1.388861 seconds
Gradient 7N: 2.949312 seconds
save 7N: 3.103391 seconds
Gradient 7n: 2.910413 seconds
save 7n: 2.183527 seconds
Gradient Amb: 2.895470 seconds
save Amb: 1.994353 seconds
Gradient LC: 2.908216 seconds
save LC: 2.047175 seconds
Gradient LRt: 2.941209 seconds
save LRt: 2.519551 seconds
Gradient Pn: 2.948350 seconds
save Pn: 2.916878 seconds
Gradient Tz: 2.911093 seconds
save Tz: 2.176856 seconds
Gradient VLL: 2.890240 seconds
save VLL: 1.759285 seconds
Gradient RMC: 2.950483 seconds
save RMC: 3.225067 seconds
Gradient SNC: 2.961968 seconds
save SNC: 3.102615 seconds
Gradient SNR: 2.945084 seconds
save SNR: 2.659436 seconds
Gradient 3N: 2.944208 seconds
save 3N: 3.154390 seconds
Gradient 4N: 2.960926 seconds
save 4N: 3.026019 seconds
Gradient Sp5I: 2.945930 seconds
save Sp5I: 2.873513 seconds
Gradient Sp5O: 2.943747 seconds
save Sp5O: 3.135227 seconds
Gradient Sp5C: 2.909517 seco

# Global align

In [8]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s 1 1 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/align_subject_brain_to_atlas_v2.py'},
#                 kwargs_list=dict(stack=all_stacks),
                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 526 seconds

align all subjects to atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 10.866593 seconds


In [15]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s 1 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains.py'},
#                 kwargs_list=dict(stack=all_stacks),
                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 277.233186 seconds


In [16]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s 1 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2.py'},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 630.123794 seconds


# Local align

In [17]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py'},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 6807 seconds

LOCAL align ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 10493.096232 seconds


In [18]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_global_to_local.py'},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 485.269291 seconds


In [19]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2_local.py'},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 190 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 622.720067 seconds


In [13]:
# Transform locally transformed volumes back to atlas space

t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_reverse_global.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 220.844733 seconds


# Snake

In [ ]:
# Snake

t = time.time()
sys.stderr.write('Snake ...')

exclude_nodes = [33]

# for stack in all_stacks:
for stack in ['MD593']:

    run_distributed4(command='%(script_path)s %(stack)s %%(name_sided)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_contour_to_scoremap.py',
                    'stack': stack},
                    kwargs_list=dict(name_sided=[convert_to_left_name(s) for s in paired_structures] + \
                                     [convert_to_right_name(s) for s in paired_structures] + \
                                     [s for s in singular_structures]),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

Snake ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']
